<a href="https://colab.research.google.com/github/briziorusso/FICOchallenge/blob/main/Copy_of_FC_Part_2c_AutoKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3: Build a neural network with AutoKeras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install autokeras

In [5]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.sparse
import os

# import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model 
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPool1D, Flatten, Input, concatenate, Dropout, Activation, BatchNormalization


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix

print('Autokeras needs TF>=2')
print('TF version:',tf.__version__)


Autokeras needs TF>=2
TF version: 2.4.1


In [4]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
cwd = os.getcwd()
print(cwd)
os.chdir(cwd)

/content


## Load prepped data
- Normal Scaling of numeric variables (prep_option = 1)
- Binning (following Rudin) and one hot encoding (prep_option = 2)
- Binning and applying WOE, calculating WOE on Rudin's bins (prep_option = 3)
- Binning and applying WOE, following Rudin (prep_option = 4)

In [6]:
data_path = "/content/drive/MyDrive/Colab Notebooks/data/heloc_dataset_v1.csv"
# ori_path = "Data/heloc_dataset_v1.csv"

CLASS = 'RiskPerformance' 

# data = pd.read_csv(ori_path)
X1 = pd.read_csv(data_path)
y = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/y_data.csv")

print('Target: Bad (y=1)')
class_names = sorted(y[CLASS].unique(),  reverse=True)
print(y[CLASS].value_counts())
y_onehot = pd.get_dummies(y[CLASS])[['Bad']]
print(np.array(np.unique(y_onehot, return_counts=True)).T)

print('X shape:',X1.shape)
X1.head()

Target: Bad (y=1)
Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64
[[   0 5000]
 [   1 5459]]
X shape: (10459, 24)


,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,MaxDelqEver,NumTotalTrades,NumTradesOpeninLast12M,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,Bad,55,144,4,84,20,3,0,83,2,3,5,23,1,43,0,0,0,33,-8,8,1,1,69
1,Bad,61,58,15,41,2,4,4,100,-7,0,8,7,0,67,0,0,0,0,-8,0,-8,-8,0
2,Bad,67,66,5,24,9,0,0,100,-7,7,8,9,4,44,0,4,4,53,66,4,2,1,86
3,Bad,66,169,1,73,28,1,1,93,76,6,6,30,3,57,0,5,4,72,83,6,4,3,91
4,Bad,81,333,27,132,12,0,0,100,-7,7,8,12,0,25,0,1,1,51,89,3,1,0,80


### Original Dataset

In [16]:
# basic data preparation
from sklearn.preprocessing import LabelEncoder
CLASS = 'RiskPerformance'

# Split X and y
X = X1.drop(columns=[CLASS])
X = X.astype(float)
y = LabelEncoder().fit_transform(y_onehot['Bad'].astype('int8'))

X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X, y, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)


from autokeras import StructuredDataClassifier
# define the search
search = StructuredDataClassifier(max_trials=100)
# perform the search
search.fit(x=X_onehot_train, y=y_onehot_train, verbose=1)


Trial 27 Complete [00h 00m 40s]
val_accuracy: 0.7090908885002136

Best val_accuracy So Far: 0.7175324559211731
Total elapsed time: 00h 17m 01s
INFO:tensorflow:Oracle triggered exit
Epoch 1/6
246/246 [==============================] - 2s 4ms/step - loss: 0.6285 - accuracy: 0.6394
Epoch 2/6
246/246 [==============================] - 1s 4ms/step - loss: 0.5707 - accuracy: 0.7118
Epoch 3/6
246/246 [==============================] - 1s 5ms/step - loss: 0.5621 - accuracy: 0.7199
Epoch 4/6
246/246 [==============================] - 1s 5ms/step - loss: 0.5544 - accuracy: 0.7237
Epoch 5/6
246/246 [==============================] - 1s 5ms/step - loss: 0.5492 - accuracy: 0.7250
Epoch 6/6
246/246 [==============================] - 1s 4ms/step - loss: 0.5437 - accuracy: 0.7303
INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [17]:
search.evaluate(X_onehot_test, y_onehot_test, verbose=1);

82/82 [==============================] - 1s 4ms/step - loss: 0.5735 - accuracy: 0.7086


In [18]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 23)                0         
_________________________________________________________________
normalization (Normalization (None, 23)                47        
_________________________________________________________________
dense (Dense)                (None, 32)                768       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

In [20]:
# save the best performing model to file
model.save('model_num_auto',save_format='tf')

INFO:tensorflow:Assets written to: model_num_auto/assets


In [21]:
# scores = model.evaluate(X_onehot_train.values, y_onehot_train)
scores = model.evaluate(X_onehot_train, y_onehot_train, verbose=0)
print("Training Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# scores = model.evaluate(X_onehot_test.values, y_onehot_test)
scores = model.evaluate(X_onehot_test, y_onehot_test, verbose = 0)
print("Test Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# confusion_matrix(y_onehot_test, model.predict_classes(X_onehot_test), sample_weight=None)

Training Set: 
accuracy: 73.33%
Test Set: 
accuracy: 70.86%


### One-hot encoded

In [ ]:
# basic data preparation
from sklearn.preprocessing import LabelEncoder
X = X_onehot.astype('int8')
y = LabelEncoder().fit_transform(y_onehot['Bad'].astype('int8'))

X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X, y, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)


from autokeras import StructuredDataClassifier
# define the search
search = StructuredDataClassifier(max_trials=15)
# perform the search
search.fit(x=X_onehot_train, y=y_onehot_train, verbose=1)
# evaluate the model
# loss, acc = search.evaluate(X_onehot_test, y_onehot_test, verbose=1)

In [ ]:
search.evaluate(X_onehot_test, y_onehot_test, verbose=0);

In [ ]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

In [ ]:
# save the best performing model to file
model.save('model_auto.h5',save_format='tf')

### WoE variables

In [7]:
data_path = "/content/drive/MyDrive/Colab Notebooks/data/WOE_Rud_data.csv"
# ori_path = "Data/heloc_dataset_v1.csv"

CLASS = 'RiskPerformance' 

# data = pd.read_csv(ori_path)
X1 = pd.read_csv(data_path)
y = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/y_data.csv")

print('Target: Bad (y=1)')
class_names = sorted(y[CLASS].unique(),  reverse=True)
print(y[CLASS].value_counts())
y_onehot = pd.get_dummies(y[CLASS])[['Bad']]
print(np.array(np.unique(y_onehot, return_counts=True)).T)

print('X shape:',X1.shape)
X1.head()

Target: Bad (y=1)
Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64
[[   0 5000]
 [   1 5459]]
X shape: (10459, 23)


,ExternalRiskEstimate_bin_WOE,MSinceOldestTradeOpen_bin_WOE,MSinceMostRecentTradeOpen_bin_WOE,AverageMInFile_bin_WOE,NumSatisfactoryTrades_bin_WOE,NumTrades60Ever2DerogPubRec_bin_WOE,NumTrades90Ever2DerogPubRec_bin_WOE,NumTotalTrades_bin_WOE,NumTradesOpeninLast12M_bin_WOE,PercentTradesNeverDelq_bin_WOE,MSinceMostRecentDelq_bin_WOE,MaxDelq2PublicRecLast12M_bin_WOE,MaxDelqEver_bin_WOE,PercentInstallTrades_bin_WOE,NetFractionInstallBurden_bin_WOE,NumInstallTradesWBalance_bin_WOE,MSinceMostRecentInqexcl7days_bin_WOE,NumInqLast6M_bin_WOE,NumInqLast6Mexcl7days_bin_WOE,NetFractionRevolvingBurden_bin_WOE,NumRevolvingTradesWBalance_bin_WOE,NumBank2NatlTradesWHighUtilization_bin_WOE,PercentTradesWBalance_bin_WOE
0,1.799,0.086,0.083,0.269,0.166,0.952,-0.021,-0.097,-0.021,1.012,-0.058,0.806,-0.147,-0.503,0.047,0.242,1.223,-0.047,-0.051,-0.088,0.034,-0.601,-0.130
1,1.799,0.549,0.083,1.238,1.999,0.952,-0.053,0.535,-0.021,-0.147,0.198,0.806,-0.147,0.161,0.047,0.256,1.223,-0.047,-0.051,-0.739,-0.188,0.601,-0.982
2,1.017,0.549,0.083,1.238,0.539,-0.021,-0.021,0.535,0.428,-0.147,0.198,-0.147,-0.147,-0.503,0.147,0.242,1.223,0.170,0.021,0.633,-0.263,-0.601,0.203
3,1.017,0.086,0.083,0.269,-0.086,-0.021,-0.021,-0.377,0.287,0.366,-0.482,0.408,-0.147,-0.145,0.363,0.313,1.223,0.471,0.021,0.633,-0.150,0.541,0.772
4,-1.094,-0.148,0.000,0.000,0.539,-0.021,-0.021,0.116,-0.021,-0.147,0.198,-0.147,-0.147,-0.620,0.363,0.242,1.223,-0.047,-0.051,0.633,-0.188,-0.601,0.203


In [ ]:
# basic data preparation
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import preprocessing
X = X1.astype('float32')
y = LabelEncoder().fit_transform(y_onehot['Bad'].astype('int8'))

X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X, y, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)


from autokeras import StructuredDataClassifier
# define the search
search = StructuredDataClassifier(max_trials=100,project_name="AKWOEdata",)
# perform the search
search.fit(x=X_onehot_train, y=y_onehot_train, verbose=1)
# evaluate the model
# loss, acc = search.evaluate(X_onehot_test, y_onehot_test, verbose=1)

Trial 1 Complete [00h 00m 21s]
val_accuracy: 0.7136363387107849

Best val_accuracy So Far: 0.7136363387107849
Total elapsed time: 00h 00m 21s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
structured_data...|True              |True              
structured_data...|False             |False             
structured_data...|2                 |2                 
structured_data...|32                |32                
structured_data...|0                 |0                 
structured_data...|32                |32                
classification_...|0                 |0                 
optimizer         |sgd               |adam              
learning_rate     |0.001             |0.001             

Epoch 1/1000
197/197 [==============================] - 2s 7ms/step - loss: 0.6834 - accuracy: 0.5710 - val_loss: 0.6830 - val_accuracy: 0.5870
Epoch 2/1000
197/197 [==============================] - 1s 6ms/step - loss: 0.6764 - accuracy: 0.5807 - val_loss: 0.

In [ ]:
search.evaluate(X_onehot_test, y_onehot_test, verbose=1);

82/82 [==============================] - 0s 1ms/step - loss: 0.5710 - accuracy: 0.7059


In [ ]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 23)                0         
_________________________________________________________________
normalization (Normalization (None, 23)                47        
_________________________________________________________________
dense (Dense)                (None, 256)               6144      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224  

In [ ]:
# save the best performing model to file
model.save('model_WOE_auto',save_format='tf')

INFO:tensorflow:Assets written to: model_WOE_auto\assets


In [ ]:
# scores = model.evaluate(X_onehot_train.values, y_onehot_train)
scores = model.evaluate(X_onehot_train, y_onehot_train, verbose=0)
print("Training Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# scores = model.evaluate(X_onehot_test.values, y_onehot_test)
scores = model.evaluate(X_onehot_test, y_onehot_test, verbose = 0)
print("Test Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# confusion_matrix(y_onehot_test, model.predict_classes(X_onehot_test), sample_weight=None)

Training Set: 
accuracy: 73.36%
Test Set: 
accuracy: 70.59%
